In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import networkx as nx
import numpy as np
import cvxpy as cp
import json
import time
from tqdm import tqdm
from pathlib import Path

from src.load_data import (
    read_metadata_networks_tntp,
    read_graph_transport_networks_tntp,
    read_traffic_mat_transport_networks_tntp,
)

from src.models import SDModel, BeckmannModel
from src.algs import subgd, ustm, frank_wolfe
from src.cvxpy_solvers import get_max_traffic_mat_mul
from src.commons import Correspondences

import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

%matplotlib inline

In [ ]:
networks_path = Path("./TransportationNetworks")

# folder = "SiouxFalls"
# net_name = "SiouxFalls_net"
# traffic_mat_name = "SiouxFalls_trips"

folder = "Anaheim"
net_name = "Anaheim_net"
traffic_mat_name = "Anaheim_trips"
    
# folder = "Barcelona"
# net_name = "Barcelona_net"
# traffic_mat_name = "Barcelona_trips"
    
net_file = networks_path / folder / f"{net_name}.tntp"
traffic_mat_file = networks_path / folder / f"{traffic_mat_name}.tntp"
graph, metadata = read_graph_transport_networks_tntp(net_file)
correspondences = read_traffic_mat_transport_networks_tntp(traffic_mat_file, metadata)
n = graph.number_of_nodes()

print(f"{graph.number_of_edges()=}, {graph.number_of_nodes()=}")

In [ ]:
model = BeckmannModel(graph, correspondences)

eps = 1e-6
mean_bw = model.graph.ep.capacities.a.mean()
mean_cost = model.graph.ep.free_flow_times.a.mean()

# cost suboptimality <= eps * (average link cost * avg bandwidth * |E| \approx total cost when beta=1)
eps_abs = eps * mean_cost * mean_bw * graph.number_of_edges()

eps_cons_abs = eps * mean_bw 
# sum of capacity violation <= eps * average link capacity
print(eps_abs, eps_cons_abs)

# Beckmann

In [ ]:
model = BeckmannModel(graph, correspondences)


times_e_ustm, flows_e_ustm, dgap_ustm, cons_log, A_log, optimal = ustm(model, eps_abs,
                                                                         max_iter=1000, stop_by_crit=True)
print(len(dgap_ustm), "shortest paths calls")


In [ ]:
times_e_fw, flows_e_fw, dgap_fw, optimal = frank_wolfe(model, eps_abs,  max_iter=1000, stop_by_crit=False)

In [ ]:
A_arr = np.array(A_log) 
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_ustm = np.abs(dgap_ustm)
dgap_fw = np.abs(dgap_fw)
plt.plot(dgap_ustm, c="C4", label="USTM")
plt.plot(dgap_fw, c="C5", label="FW")
plt.plot(np.ones(max(dgap_ustm.size, dgap_fw.size)) * eps_abs, label="eps_abs")
plt.yscale("log")
# plt.ylim(None, dgap_ustm.max() * 2)
plt.title("abs(dgap)")
plt.legend()


# SD

In [ ]:
# %%time

# for Anaheim 18 min on my laptop (for np.ones matrix ~3 min, omg. maybe scaling impacts?). result \approx 0.529
# sd_max_traffic_mul = get_max_traffic_mat_mul(graph, correspondences.traffic_mat, solver=cp.SCS)
# sd_max_traffic_mul = 0.529

# for SiouxFalls
# sd_max_traffic_mul = get_max_traffic_mat_mul(graph, correspondences.node_traffic_mat, solver=cp.ECOS, 
#                                              max_iters=1000, verbose=False)

print(f"{sd_max_traffic_mul=},")

corrs = Correspondences(traffic_mat=correspondences.traffic_mat * sd_max_traffic_mul * 0.9,
                        node_traffic_mat=correspondences.node_traffic_mat * sd_max_traffic_mul * 0.9,
                        sources=correspondences.sources,
                        targets=correspondences.targets)

model = SDModel(graph, corrs)
# model = SDModel(graph, traffic_mat * sd_max_traffic_mul * 1.1)  # you should see that constraints error does not converge

## cvxpy edge-based

## Ustm

In [ ]:
%%time

times_e_ustm, flows_e_ustm, dgap_log, cons_log, A_log, optimal = ustm(model, eps_abs, eps_cons_abs,
                                                                         max_iter=1000, stop_by_crit=True)
print(len(dgap_log), "iters")


In [ ]:
A_arr = np.array(A_log) 
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_abs = np.abs(dgap_log)
plt.plot(dgap_abs, c="C4", label="USTM")
plt.plot(np.ones(dgap_abs.size) * eps_abs, label="eps_abs")
plt.yscale("log")
plt.ylim(None, dgap_abs.max() * 2)
plt.title("abs(dgap)")
plt.legend()

plt.subplot(122)
plt.plot(np.ones(len(cons_log)) * eps_cons_abs, label="eps_cons_abs")
plt.plot(cons_log, c="C4", label="USTM")
# plt.plot(4 * R / A_arr + (2 * eps_abs / A_arr)) # actually, it is bound for 2-norm, while cons_log is 1-norm
# plt.ylim(np.min(cons_log) / 1.5, np.max(cons_log) * 2)
plt.title("constraint violation")
plt.yscale("log")
plt.legend()

print("average dgap:", np.mean(dgap_log))

In [ ]:
bws = model.graph.ep.capacities.a
order = bws.argsort()

plt.plot(bws[order], label="capacity", c='C2')
# plt.plot(flows_cvx[order], label="cvxpy", c="C1")
plt.plot(flows_e_ustm[order], label="ustm", c="C4")

plt.legend()

# Subgd

In [ ]:
%%time

R = 10
times_e_subgd, flows_e_subgd, dgap_log, cons_log, optimal = subgd(model, R=10 * R, eps_abs=eps_abs, 
                                                                   eps_cons_abs=eps_cons_abs, max_iter=1000)
R = np.linalg.norm(dual_costs)
print(len(dgap_log), "iters")

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
dgap_abs = np.abs(dgap_log)
plt.plot(np.ones(dgap_abs.size) * eps_abs, label="eps_abs")
plt.plot(dgap_abs, c="C3", label="subgd")
plt.legend()
plt.yscale("log")

plt.title("abs(dgap)")
plt.subplot(122)
plt.plot(np.ones(len(cons_log)) * eps_cons_abs, label="eps_cons_abs")
plt.plot(cons_log, c="C3", label="subgd")
plt.legend()
plt.title("constraint violation l1 norm")
plt.yscale("log")

print("average dgap:", np.mean(dgap_log))

In [ ]:
bws = model.graph.ep.capacities.a
order = bws.argsort()

plt.plot(bws[order], label="capacity", c='C2')
# plt.plot(flows_cvx[order], label="cvxpy", c="C1")
plt.plot(flows_e_subgd[order], label="subgd", c="C4")

plt.legend()